# Econometric Feature Creation

In [2]:
#imports 
import pandas as pd
import numpy as np
import os
import statsmodels.formula.api as sm
import pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange, tqdm
import statsmodels.api as sm
import networkx as nx
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold

In [5]:
import pandas as pd
import requests

def get_sitc_codes():
    # URL of the JSON file
    url = 'https://comtradeapi.un.org/files/v1/app/reference/S4.json'

    try:
        # Send a GET request to the URL and fetch the data
        response = requests.get(url)
        response.raise_for_status()  # Check that the request was successful
        
        # Load the JSON data
        data = response.json()

        # Since the JSON data might be nested, use json_normalize with appropriate arguments
        if isinstance(data, list):
            # If the top level is a list
            df = pd.json_normalize(data)
        else:
            # If the top level is a dictionary
            # Identify the key that holds the main data (adjust the path as necessary)
            main_data_key = 'results'  # Adjust this based on the actual structure
            df = pd.json_normalize(data[main_data_key])

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
    except KeyError as e:
        print(f"Error processing JSON structure: {e}")

    return df

In [6]:
#we mimic 4-class system from World Bank -- 4 growth categoris: 0 - L, 1 - LM, 2 - UM, 3 - U
def growth_categories(x, growth_list):
    threshold_high = np.percentile(growth_list, 75)
    threshold_low = np.percentile(growth_list, 25)
    threshold_mid = np.percentile(growth_list, 50)
    
    if(x > threshold_high): return 3
    elif(x < threshold_low): return 0
    elif(x < threshold_mid): return 1
    else: return 2

In [5]:
class CreateFeatures:
    """
    We define a class which builds the feature dataframe 
    """
    
    def __init__(self, year = 1962, data_dir = "../data/"):
        self.year = year
        self.data_dir = data_dir
        
    def prepare_econ_features(self, filter_gdp = True):
        
        #DATA IMPORT
        #import dictionary with all features from WB
        with open(self.data_dir + 'all_wb_indicators.pickle', 'rb') as handle:
            features_dict = pkl.load(handle)
            
        self.feature_list = list(features_dict.keys())[1:]
        #import list of all features we want to select for

        #look up each of the features -- add country feature in that year 
        i = 0
        for feature in self.feature_list:
            #find dataframe corresponding to specific feature name
            df = features_dict[feature]
            
            if (i == 0):
                self.features = df[["economy", "YR" + str(self.year)]]
            else: 
                self.features = pd.merge(self.features, 
                                            df[["economy", "YR" + str(self.year)]],
                                            on = "economy", how = "outer")
            self.features.rename(columns = {"YR" + str(self.year): feature}, inplace = True)
            i = i+1
        
        #prepare GDP feature
        self.gdp_growth = features_dict['NY.GDP.MKTP.KD.ZG']
        cols = list(self.gdp_growth.columns.copy())
        cols.remove("economy")
        self.gdp_growth["country_sd"] = self.gdp_growth[cols].std(axis=1)
        #select potential variables 
        self.gdp_growth["prev_gdp_growth"] = self.gdp_growth["YR" + str(self.year-1)]
        self.gdp_growth["current_gdp_growth"] = self.gdp_growth["YR" + str(self.year)]
        #we eliminate countries that are too volatile in growth -- probably an indicator that growth estimates are inaccurate
        self.gdp_growth = self.gdp_growth[["economy", "prev_gdp_growth",
                                "current_gdp_growth"]].dropna()
        
        #combine GDP and other features
        self.features = pd.merge(self.gdp_growth, self.features,
                                   on = "economy", how = "left")
        #we only keep countries where we observe GDP growth -- otherwise nothing to predict
        #we keep countries where other features may be missing -- and fill NAs with 0 
        self.features.rename(columns = {"economy": "country_code"}, inplace = True)
        
    def prepare_network_features(self):
        """
        We create an initial, import-centric trade link pandas dataframe for a given year
        """
        #get product codes
        data_dict = get_sitc_codes()
        data_cross = []
        i = 0
        for item_def in list(data_dict["text"]):
            if(i >= 2):
                data_cross.append(item_def.split(" - ", 1))
            i = i+1

        self.product_codes = pd.DataFrame(data_cross, columns = ['code', 'product'])
        self.product_codes["sitc_product_code"] = self.product_codes["code"]
        
        #get country codes
        self.country_codes = pd.read_excel(self.data_dir + "ISO3166.xlsx")
        self.country_codes["location_code"] = self.country_codes["Alpha-3 code"]
        self.country_codes["partner_code"] = self.country_codes["Alpha-3 code"]
        self.country_codes["country_i"] = self.country_codes["English short name"]
        self.country_codes["country_j"] = self.country_codes["English short name"]
        
        #get trade data for a given year
        trade_data = pd.read_stata(self.data_dir + "country_partner_sitcproduct4digit_year_"+ str(self.year)+".dta") 
        #merge with product / country descriptions
        trade_data = pd.merge(trade_data, self.country_codes[["location_code", "country_i"]],on = ["location_code"])
        trade_data = pd.merge(trade_data, self.country_codes[["partner_code", "country_j"]],on = ["partner_code"])
        trade_data = pd.merge(trade_data, self.product_codes[["sitc_product_code", "product"]], 
                              on = ["sitc_product_code"])
        ###select level of product aggregation
        trade_data["product_category"] = trade_data["sitc_product_code"].apply(lambda x: x[0:1])
        #trade_data = trade_data[trade_data["product_category"] == "1"]
        
        #keep only nodes that we have features for
        trade_data = trade_data[trade_data["location_code"].isin(self.features["country_code"])]
        trade_data = trade_data[trade_data["partner_code"].isin(self.features["country_code"])]
        
        if (len(trade_data.groupby(["location_code", "partner_code", "sitc_product_code"])["import_value"].sum().reset_index()) != len(trade_data)):
            print("import, export, product combination not unique!")
        self.trade_data1 = trade_data
        #from import-export table, create only import table
        #extract imports
        imports1 = trade_data[['location_id', 'partner_id', 'product_id', 'year',
               'import_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
               'sitc_product_code', 'country_i', 'country_j', 'product', "product_category"]]
        imports1 = imports1[imports1["import_value"] != 0]
        #transform records of exports into imports
        imports2 = trade_data[['location_id', 'partner_id', 'product_id', 'year',
               'export_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
               'sitc_product_code', 'country_i', 'country_j', 'product', "product_category"]]
        imports2["temp1"] = imports2['partner_code']
        imports2["temp2"] = imports2['location_code']

        imports2['location_code'] = imports2["temp1"]
        imports2['partner_code'] = imports2["temp2"]
        imports2["import_value"] = imports2["export_value"]
        imports2 = imports2[imports2["import_value"] != 0]
        imports2 = imports2[['location_id', 'partner_id', 'product_id', 'year',
               'import_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
               'sitc_product_code', 'country_i', 'country_j', 'product', "product_category"]]
        
        imports_table = pd.concat([imports1, imports2]).drop_duplicates()
        
        #rename columns for better clarity
        imports_table["importer_code"] = imports_table["location_code"]
        imports_table["exporter_code"] = imports_table["partner_code"]
        imports_table["importer_name"] = imports_table["country_i"]
        imports_table["exporter_name"] = imports_table["country_j"]
        
        cols = ["importer_code", "exporter_code", "importer_name", "exporter_name",
               'product_id', 'year', 'import_value', 'sitc_eci', 'sitc_coi',
               'sitc_product_code', 'product', "product_category"]
        imports_table = imports_table[cols]
        
        exporter_total = imports_table.groupby(["exporter_code"])["import_value"].sum().reset_index()
        exporter_total = exporter_total.rename(columns = {"import_value": "export_total"})
        
        importer_total = imports_table.groupby(["importer_code"])["import_value"].sum().reset_index()
        importer_total = importer_total.rename(columns = {"import_value": "import_total"})
        
        ##### COMPUTE CENTRALITY FOR COUNTRY
        #sum imports across all products between countries into single value 
        imports_table_grouped = imports_table.groupby(["importer_code", "exporter_code"])["import_value"].sum().reset_index()
        imports_table_grouped = pd.merge(imports_table_grouped, importer_total, on = "importer_code")
        imports_table_grouped["import_fraction"] = imports_table_grouped["import_value"]\
                        /imports_table_grouped["import_total"]*100
        
        self.trade_data = imports_table_grouped
        
        #filter features and nodes to ones that are connected to others in trade data
        list_active_countries = list(set(list(self.trade_data ["importer_code"])+\
                        list(self.trade_data ["exporter_code"])))
        self.features = self.features[self.features["country_code"].isin(list_active_countries)].reset_index()
        self.features["node_numbers"] = self.features.index
        
        G=nx.from_pandas_edgelist(self.trade_data, 
                          "exporter_code", "importer_code", create_using = nx.DiGraph())
        
        self.G = G
        self.centrality_overall= nx.eigenvector_centrality(G, max_iter= 10000) 
        self.centrality_overall = pd.DataFrame(list(map(list, self.centrality_overall.items())), 
                                               columns = ["country_code", "centrality_overall"])
        G=nx.from_pandas_edgelist(self.trade_data, 
                          "exporter_code", "importer_code", ["import_fraction"])
        weighted_centrality = nx.eigenvector_centrality(G, weight = "import_fraction", max_iter= 10000) 
        weighted_centrality  = pd.DataFrame(list(map(list, weighted_centrality.items())), 
                                               columns = ["country_code", "weighted_centrality"])
        self.centrality_overall = pd.merge(self.centrality_overall, weighted_centrality, on = "country_code")
        
                               
        ##### COMPUTE CENTRALITY FOR COUNTRY IN PRODUCT CATEGORIES

        #sum imports across all products between countries into single value 
        imports_table_grouped = imports_table.groupby(["importer_code", "exporter_code"])["import_value"].sum().reset_index()
        products_grouped = imports_table.groupby(["product_category"])["import_value"].sum().reset_index()
        products_grouped = products_grouped.rename(columns = {"import_value": "import_product_total"})
        
        #sum exports in each category 
        self.export_types = imports_table.groupby(["importer_code", "exporter_code", "product_category"])["import_value"].sum().reset_index()
        self.export_types = pd.merge(products_grouped, self.export_types, on = "product_category")
        
        self.export_types["product_export_fraction"] = self.export_types["import_value"]\
                                                    /self.export_types["import_product_total"]*100
        
        list_products = list(set(self.export_types["product_category"]))
        
        i = 0 
        for product in list_products:
            
            temp = self.export_types[self.export_types["product_category"] == product].copy()
            
            G_w=nx.from_pandas_edgelist(temp, 
                "exporter_code", "importer_code", ["product_export_fraction"], create_using = nx.DiGraph())
            centrality_product_w = nx.eigenvector_centrality(G_w, weight = "product_export_fraction", 
                                                           max_iter= 10000)

            G=nx.from_pandas_edgelist(temp,"exporter_code", "importer_code", create_using = nx.DiGraph())
            centrality_product = nx.eigenvector_centrality(G,max_iter= 10000)

            if(i == 0):
                self.centrality_product = pd.DataFrame(list(map(list, centrality_product.items())), 
                                               columns = ["country_code", "prod_" + product])
                

            else: 
                self.centrality_product = pd.merge(self.centrality_product, 
                                               pd.DataFrame(list(map(list, centrality_product.items())), 
                                               columns = ["country_code", "prod_" + product]), 
                                                  on = "country_code")
                
            self.centrality_product = pd.merge(self.centrality_product, 
                                               pd.DataFrame(list(map(list, centrality_product_w.items())), 
                                               columns = ["country_code", "prod_w_" + product]), 
                                                  on = "country_code")
            
            i = i+1         
    
    def combine_normalize_features(self):
        
        self.combined_features = pd.merge(self.features, self.centrality_overall,on = "country_code")
        self.combined_features = pd.merge(self.combined_features, self.centrality_product,on = "country_code")
        #step eliminates NA and nodes that are not in graph, since they will have NA for graph features
        self.combined_features = self.combined_features.drop(columns = ["index"])
        #filter both trade data and features data to same subset of countries
        self.combined_features = self.combined_features[\
                                self.combined_features.country_code.isin(self.trade_data.importer_code)|\
                                self.combined_features.country_code.isin(self.trade_data.exporter_code)]
        self.trade_data = self.trade_data[\
                          self.trade_data.importer_code.isin(self.combined_features.country_code)&\
                          self.trade_data.exporter_code.isin(self.combined_features.country_code)]
        
        features_to_norm = list(self.combined_features.columns.copy())
        non_norm = ["country_code", "node_numbers"]
        cols_insufficient_data = list(self.combined_features.loc[:, self.combined_features.nunique() < 2].columns.copy())
        non_norm.extend(cols_insufficient_data)
 
        features_to_norm = [x for x in features_to_norm if x not in non_norm]
        scaler = StandardScaler()
        #we preserve NAs in the scaling
        self.combined_features[features_to_norm] = scaler.fit_transform(self.combined_features[features_to_norm])
        self.combined_features.fillna(0, inplace = True) #we fill NA after scaling 
        #check that feature has at least 20% coverage in a given year -- otherwise set to NA
        for feature in self.feature_list:
            coverage = len(self.combined_features[self.combined_features[feature] != 0])/len(self.combined_features)
            if(coverage <= 0.20): self.combined_features[feature] = 0

In [11]:
data_dict = {}

#GDP data goes up to 2018 -- trange up to 2019 spans up to 2018
for year in trange(1962, 2019):
    trade = CreateFeatures(year = year)
    trade.prepare_econ_features()
    trade.prepare_network_features()
    trade.combine_normalize_features()
    
    data_dict[year] = trade

  0%|          | 0/57 [00:00<?, ?it/s]/var/folders/qh/0wyfgfcs3399gr_m2wrrtnzc0000gn/T/ipykernel_1192/2289272989.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.features.rename(columns = {"YR" + str(self.year): feature}, inplace = True)
  2%|▏         | 1/57 [00:31<29:02, 31.12s/it]/var/folders/qh/0wyfgfcs3399gr_m2wrrtnzc0000gn/T/ipykernel_1192/2289272989.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.features.rename(columns = {"YR" + str(self.year): feature}, inplace = True)
  4%|▎         | 2/57 [01:03<29:02, 31.68s/it]/var/folders/qh/0wyfgfcs3399gr_m2wrrtnzc0000gn/T/ipykernel_1192/2289272989.py:

In [12]:
data_dict[2000].combined_features

,country_code,prev_gdp_growth,current_gdp_growth,AG.CON.FERT.PT.ZS,AG.CON.FERT.ZS,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,...,prod_7,prod_w_7,prod_5,prod_w_5,prod_8,prod_w_8,prod_4,prod_w_4,prod_2,prod_w_2
0,ABW,-0.490291,0.848490,0.000000,0.000000,-0.380982,-1.253366,-0.329404,-0.827150,-0.253380,...,-1.414827,-0.313809,-1.465817,-0.364846,-0.969262,-0.258203,-1.136650,-0.338097,-1.287930,-0.297794
1,AGO,-0.251615,-0.294556,0.000000,-0.505692,0.299063,-0.056515,-0.196037,-0.199203,-0.882967,...,-0.380104,-0.303504,-0.427126,-0.364506,-0.414681,-0.275268,-0.612258,-0.335358,-0.472332,-0.293682
2,ALB,2.457652,0.680522,0.000000,-0.406316,-0.364444,0.127504,-0.303780,-0.183085,0.468716,...,-0.516679,-0.316662,-0.263035,-0.367098,-0.385573,-0.279064,-0.324549,-0.329935,-0.513299,-0.301128
3,AND,0.233501,-0.175856,0.000000,0.000000,-0.380678,0.451277,-0.329448,-0.855770,-0.903124,...,-1.059324,-0.316697,-1.575202,-0.369057,-0.214850,-0.273801,-1.081020,-0.338717,-1.155239,-0.302508
4,ARE,-0.069285,1.659332,-0.322823,1.316045,-0.373090,-1.407000,-0.326823,-0.843295,-0.995907,...,0.494390,-0.065853,0.620845,-0.241015,0.571215,-0.074475,0.782526,-0.299515,0.814640,-0.226703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,WSM,-0.250617,0.216191,0.000000,0.000000,-0.380316,-0.989726,-0.328870,-0.617359,-0.699210,...,-1.591910,-0.314139,-1.567570,-0.370898,-1.790937,-0.282132,-1.282345,-0.340369,-1.331202,-0.304292
178,YEM,0.151649,0.489018,0.000000,-0.476489,-0.038247,0.266235,-0.260763,-0.590555,-0.845361,...,-0.380955,-0.308096,-0.089159,-0.362013,-0.377225,-0.278477,-0.524392,-0.326745,-0.344331,-0.299589
179,ZAF,-0.196336,-0.007571,-0.236817,-0.340104,1.039992,1.891255,0.284936,0.239064,-0.233512,...,1.760814,-0.072691,1.572291,0.006781,1.566668,-0.117665,1.584234,-0.144677,1.243733,-0.153298
180,ZMB,0.372922,-0.083409,2.299392,-0.472405,-0.055205,-0.390210,-0.204222,0.198601,-0.783033,...,0.205731,-0.316253,-0.103127,-0.366304,-0.207051,-0.280231,-0.727763,-0.336402,-0.187047,-0.301829


In [13]:
with open("../data/features_dict.pkl", "wb") as f:
    pkl.dump(data_dict, f)

In [6]:
with open("../data/features_dict.pkl", "rb") as f:
    data_dict = pkl.load(f)

In [7]:
# Initialize variables to keep track of the DataFrame with the most rows
max_rows = 0
df_with_max_rows = None

# Iterate over the dictionary
for key, df in data_dict.items():
    # If this DataFrame has more rows than the current maximum, update the maximum and the DataFrame
    if len(df.combined_features) > max_rows:
        max_rows = len(df.combined_features)
        df_with_max_rows = df.combined_features
        max_year = key

In [8]:
# Compute the variance of each column
numeric_columns = df_with_max_rows.drop(['country_code'], axis=1)
variances = numeric_columns.var()

# Find columns with variance less than 0.1 (this is the threshold, adjust as needed)
columns_to_drop = variances[variances < 0.1].index
filtered_df = df_with_max_rows.drop(columns_to_drop, axis=1)

In [9]:
# Iterate over the dictionary
for key, df in data_dict.items():
    df.combined_features = df.combined_features.drop(columns_to_drop, axis=1)

## Mutual Information Selection

In [ ]:
X_train = filtered_df.drop(['country_code','current_gdp_growth'], axis = 1)
Y_train = filtered_df['current_gdp_growth']

In [58]:
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectPercentile

mutual_info = mutual_info_regression(X_train, Y_train)

mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

selected_top_columns = SelectPercentile(mutual_info_regression, percentile=10)
selected_top_columns.fit(X_train, Y_train)
selected_top_columns.get_support()

array([ True, False, False, ..., False, False, False])

In [59]:
columns = X_train.columns[selected_top_columns.get_support()]
X_train = X_train[columns]

In [64]:
column_list = list(columns)
column_list.append('current_gdp_growth')
column_list.append('country_code')

In [66]:
for key, df in data_dict.items():
    df.combined_features = df.combined_features[column_list]

In [71]:
columns_all_zeros = []
for key, df in data_dict.items():
    zero_columns = [col for col in df.combined_features.columns if (df.combined_features[col] == 0).all()]

    # Iterate over list1
    for item in zero_columns:
        # If the item is not in list2, add it
        if item not in columns_all_zeros:
            columns_all_zeros.append(item)


In [75]:
for key, df in data_dict.items():
    df.combined_features = df.combined_features.drop(columns_all_zeros, axis=1)

In [77]:
data_dict[1962].combined_features

,prev_gdp_growth,FM.AST.PRVT.ZG.M3,FM.LBL.BMNY.ZG,IT.MLT.MAIN.P2,NE.CON.PRVT.KD.ZG,NE.CON.PRVT.PC.KD.ZG,NE.CON.TOTL.KD.ZG,NE.GDI.FTOT.KD.ZG,NE.GDI.TOTL.KD.ZG,NE.IMP.GNFS.KD.ZG,...,SP.POP.7074.FE.5Y,SP.POP.7074.MA.5Y,SP.POP.7579.MA.5Y,SP.POP.80UP.MA.5Y,SP.POP.DPND.OL,SP.POP.DPND.YG,SP.POP.GROW,SP.URB.GROW,current_gdp_growth,country_code
0,0.190007,0.145456,-0.309841,-0.111774,0.000000,0.000000,-1.024184,-1.066195,-0.806555,-0.258654,...,0.188540,0.346821,-0.093827,-0.580121,-0.024536,-1.192493,-0.627666,-0.906953,-1.112815,ARG
1,-0.292556,0.145012,-0.424987,1.301831,-0.340188,-0.282538,-0.246697,-0.540698,0.000000,-0.949600,...,1.366470,1.289176,1.081964,0.970590,1.229090,-1.161966,0.184470,-0.696583,-0.714648,AUS
2,0.208053,0.000000,0.000000,0.097787,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.652608,2.290681,2.279018,1.948592,2.477550,-1.855186,-1.645028,-1.742712,-0.463491,AUT
3,0.116371,0.000000,0.000000,0.417067,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.355234,2.329008,2.448650,2.382717,2.419029,-1.744078,-1.860075,-1.820086,0.011966,BEL
4,-0.184642,0.000000,0.000000,-0.685382,-0.913001,-0.753715,-0.700270,0.000000,-0.447176,-0.038399,...,0.216401,0.317638,0.277749,0.234998,0.401940,-0.064347,-0.620756,1.736960,-1.590323,BEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,-0.509915,0.146601,-0.352908,-0.611217,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.185401,-0.427716,-0.470985,-0.241155,-0.142476,0.354390,0.462587,0.089780,0.078634,TUR
72,-0.290855,0.146321,-0.526697,-0.129166,0.000000,0.000000,0.102288,-0.490386,0.000000,0.391464,...,0.947874,1.081684,1.100397,1.256356,0.978531,-1.451146,-1.043111,-1.345748,-1.246832,URY
73,-0.322484,0.145460,-0.371624,2.761470,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.428942,1.687666,1.564861,1.494738,1.606441,-1.045552,-0.719038,-1.051855,0.176675,USA
74,-0.176246,-6.855651,-0.759620,-0.495877,-0.245292,0.000000,-0.328785,-0.399401,0.120419,0.000000,...,-0.922368,-0.900646,-0.821015,-0.755621,-0.929226,1.013644,1.243462,0.486535,0.627946,VEN


In [78]:
with open("../data/filtered_features_dict.pkl", "wb") as f:
    pkl.dump(data_dict, f)

## Random Subset Selection

In [25]:
data_dict[1962].combined_features.iloc[:,-21:]

,weighted_centrality,prod_6,prod_w_6,prod_1,prod_w_1,prod_0,prod_w_0,prod_9,prod_w_9,prod_3,...,prod_7,prod_w_7,prod_5,prod_w_5,prod_8,prod_w_8,prod_4,prod_w_4,prod_2,prod_w_2
0,-0.576489,0.499604,-0.032686,0.577605,-0.078327,0.173909,-0.407186,1.047343,-0.250979,1.383193,...,0.845193,0.953212,0.955955,-0.058250,0.823576,-0.109698,0.047138,-0.495751,0.675765,-0.319671
1,-0.289337,0.227220,0.704168,0.432205,1.279786,0.576321,-0.295091,0.244263,-0.114330,0.033907,...,0.110394,0.973730,0.359377,0.976446,0.403105,1.163529,0.559864,-0.308440,0.575281,0.126215
2,-0.806198,-0.248175,-0.260587,-0.181334,-0.100772,0.143657,-0.326047,0.127504,-0.261322,0.395508,...,-0.450748,-0.346783,-0.010160,-0.377199,-0.182387,-0.230070,0.569347,0.717353,0.249491,-0.185634
3,0.177899,2.000982,0.580241,2.547139,0.949078,2.140013,1.008893,1.418409,-0.216389,1.337127,...,1.697043,0.638313,1.457639,1.184763,1.000250,1.051989,2.054749,0.627368,1.948377,1.048534
4,-0.720938,-1.057364,-0.394583,-0.665602,-0.397013,-0.836336,-0.507175,-1.163366,-0.302108,-0.601007,...,-0.988531,-0.541049,-1.122434,-0.648742,-1.110936,-0.445013,-1.200309,-0.539445,-1.121349,-0.389850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,-0.164070,0.016782,-0.188413,-0.467708,-0.412145,-0.474310,0.146435,-0.200846,-0.186034,0.574126,...,0.166053,-0.113035,0.131481,-0.270442,0.070701,-0.342667,0.884631,0.885479,0.337973,-0.353501
72,-0.145000,-0.165810,-0.365474,0.143974,0.034505,-0.117380,-0.483092,-0.092322,-0.272072,-0.377495,...,-0.204803,-0.387454,0.211505,-0.530099,0.050095,-0.387218,-0.681981,-0.532784,0.092448,-0.366246
73,0.721071,2.636640,6.815271,2.580423,6.116480,2.207509,3.836014,3.362049,7.026928,2.306715,...,2.493777,3.395988,2.454607,3.855807,3.286510,3.845328,2.545039,1.711141,2.019389,5.311858
74,0.543287,0.548058,0.338022,0.886643,-0.300293,0.752783,0.186351,1.404265,0.072024,0.214926,...,0.633133,0.367710,0.624675,0.283557,0.696338,0.250895,0.746932,0.202425,0.796400,-0.259636


In [29]:
cols_keep = list(data_dict[1962].combined_features.iloc[:,-21:].columns) + ['country_code', 'current_gdp_growth','prev_gdp_growth']

In [30]:
# Iterate over the dictionary
for key, df in data_dict.items():
    # Get column names for columns 100 through 500
    col_range = df.combined_features.columns[100:501]

    # Combine the lists
    all_cols = cols_keep + col_range.tolist()

    # Select the columns from the dataframe
    df.combined_features = df.combined_features[all_cols]

In [32]:
data_dict[1962].combined_features

,weighted_centrality,prod_6,prod_w_6,prod_1,prod_w_1,prod_0,prod_w_0,prod_9,prod_w_9,prod_3,...,NV.AGR.TOTL.KN,NV.AGR.TOTL.ZS,NV.FSM.TOTL.CN,NV.FSM.TOTL.KN,NV.IND.EMPL.KD,NV.IND.MANF.CD,NV.IND.MANF.CN,NV.IND.MANF.KD,NV.IND.MANF.KD.ZG,NV.IND.MANF.KN
0,-0.576489,0.499604,-0.032686,0.577605,-0.078327,0.173909,-0.407186,1.047343,-0.250979,1.383193,...,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.289337,0.227220,0.704168,0.432205,1.279786,0.576321,-0.295091,0.244263,-0.114330,0.033907,...,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
2,-0.806198,-0.248175,-0.260587,-0.181334,-0.100772,0.143657,-0.326047,0.127504,-0.261322,0.395508,...,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.177899,2.000982,0.580241,2.547139,0.949078,2.140013,1.008893,1.418409,-0.216389,1.337127,...,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.720938,-1.057364,-0.394583,-0.665602,-0.397013,-0.836336,-0.507175,-1.163366,-0.302108,-0.601007,...,0.000000,0.724287,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,-0.164070,0.016782,-0.188413,-0.467708,-0.412145,-0.474310,0.146435,-0.200846,-0.186034,0.574126,...,0.000000,1.221266,0,0,0,-0.161398,-0.590336,0.000000,0.000000,0.000000
72,-0.145000,-0.165810,-0.365474,0.143974,0.034505,-0.117380,-0.483092,-0.092322,-0.272072,-0.377495,...,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
73,0.721071,2.636640,6.815271,2.580423,6.116480,2.207509,3.836014,3.362049,7.026928,2.306715,...,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
74,0.543287,0.548058,0.338022,0.886643,-0.300293,0.752783,0.186351,1.404265,0.072024,0.214926,...,-0.291651,0.000000,0,0,0,0.000000,0.000000,0.000000,0.050742,-0.380044


In [33]:
with open("../data/random_features_dict.pkl", "wb") as f:
    pkl.dump(data_dict, f)